<a href="https://colab.research.google.com/github/marendtz/TransformerNER/blob/master/code_snippets_XLMRforTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preparations**

In [106]:
#clean up model
import shutil
try:
  shutil.rmtree('/content/xlm-roberta-base-finetuned-panx-en')
except:
  print("no such folder")

In [7]:
import getpass
#get github access token for collab
token = getpass.getpass()

··········


In [8]:
%cd /content/
%rm -r /content/*
%ls -la
%pwd

/content
total 16
drwxr-xr-x 1 root root 4096 Apr 14 12:45 ./
drwxr-xr-x 1 root root 4096 Apr 14 09:21 ../
drwxr-xr-x 4 root root 4096 Apr 12 13:33 .config/


'/content'

In [9]:
# clone github repo
!git clone https://{token}@github.com/marendtz/TransformerNER.git

Cloning into 'TransformerNER'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 57 (delta 24), reused 18 (delta 4), pack-reused 0
Unpacking objects: 100% (57/57), 113.65 KiB | 270.00 KiB/s, done.


In [10]:
%pip install -r /content/TransformerNER/requirements.txt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
from collections import defaultdict
import pandas as pd
import numpy as np
from datasets import load_dataset_builder, get_dataset_config_names, load_dataset, DatasetDict

from transformers import AutoTokenizer, TrainingArguments, DataCollatorForTokenClassification, Trainer
import nltk.data
nltk.download('punkt')

import torch.nn as nn
from transformers import XLMRobertaConfig, AutoConfig, XLMRobertaForTokenClassification
from transformers.modeling_outputs import TokenClassifierOutput

from huggingface_hub import notebook_login
from seqeval.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report
from seqeval.scheme import IOB2


from torch.nn.functional import cross_entropy
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Get test data and tokenise as example**

In [41]:
# setup for tokenization
xlmr_model_name = "xlm-roberta-base"
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

In [42]:
# get company text 
with open('/content/TransformerNER/data/adidas_2022.txt', 'r') as file:
    data_text = file.read()
# split company text into sentenques: str --> list[string]
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentence_list=tokenizer.tokenize(data_text)
print(sentence_list)
print(sentence_list[0])

['Our commitment to sustainability is embedded into how we have done business for over two decades.', 'It is rooted in our purpose ‘Through sport, we have the power to change lives.’ In 2021, we have doubled\ndown on our commitment to sustainability and defined a roadmap for 2025 and beyond that allows us to\ncreate a positive impact across relevant areas, always focusing on the most material topics – for us and\nour stakeholders.', 'We will continue to move to a comprehensive, consumer-facing sustainable article\noffering at scale, expand our circular services, and work toward achieving climate neutrality (CO2e) across\nour entire value chain.', 'We continue to empower our employees to become sustainability ambassadors,\njust as we invite consumers globally to engage and connect with us on the topic of sustainability.', 'Lastly, we\naim to uphold the highest social compliance standards in our supply chain.', 'We believe that moving toward achieving the targets we have defined for 2025

In [43]:
for sentence in sentence_list:
  xlmr_tokens = xlmr_tokenizer(sentence).tokens()
  print(xlmr_tokens)

['<s>', '▁Our', '▁commitment', '▁to', '▁sustain', 'ability', '▁is', '▁', 'embe', 'dded', '▁into', '▁how', '▁we', '▁have', '▁done', '▁business', '▁for', '▁over', '▁two', '▁de', 'cade', 's', '.', '</s>']
['<s>', '▁It', '▁is', '▁root', 'ed', '▁in', '▁our', '▁purpose', '▁‘', 'Th', 'r', 'ough', '▁sport', ',', '▁we', '▁have', '▁the', '▁power', '▁to', '▁change', '▁lives', '.', '’', '▁In', '▁2021', ',', '▁we', '▁have', '▁double', 'd', '▁down', '▁on', '▁our', '▁commitment', '▁to', '▁sustain', 'ability', '▁and', '▁define', 'd', '▁a', '▁road', 'map', '▁for', '▁2025', '▁and', '▁beyond', '▁that', '▁allows', '▁us', '▁to', '▁create', '▁a', '▁positive', '▁impact', '▁across', '▁relevant', '▁areas', ',', '▁always', '▁focus', 'ing', '▁on', '▁the', '▁most', '▁material', '▁topic', 's', '▁–', '▁for', '▁us', '▁and', '▁our', '▁stake', 'holder', 's', '.', '</s>']
['<s>', '▁We', '▁will', '▁continue', '▁to', '▁move', '▁to', '▁a', '▁comprehensive', ',', '▁consumer', '-', 'fac', 'ing', '▁sustainable', '▁article', 

In [44]:
xlmr_tokenizer.encode(sentence_list[-1], return_tensors="pt")


tensor([[     0,    360,  66044,      4,    642,  45367, 188926,      4, 177399,
              4,    136,  51359,    111, 131703,    136,  38109,  54940,  31461,
             53,  24365,    136, 133325,    390,   4755,    214,    678, 105207,
          81450,      7,   6044,    237,     70,   8357,    239,  38648, 173101,
             26,      7,     15,    371, 107078,     26,     16,    204,   6766,
           3055,  27985,     26,   1528,      4,    237,   5299,    237,    678,
             70,  14098, 145704,   8357, 173101,    100, 180809,     15,    371,
          17780,    594,     26,     16,    678,     70, 151814,    111,     22,
              7,  53089,    450,     70,  18940,  38109,    111, 110613,    136,
          43017, 133325,    621,   1257, 100038,     23,     70, 144585, 100677,
         121293,      5,      2]])

**Get already defined model class XLMRobertaForTokenClassification and supply own config + test output with random weights**

In [45]:
# define model

# get info from dataset later used for training our model
# load dataset to get the tags used, so we later take the same...
lang = "en"
ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
tags = ds["train"].features["ner_tags"].feature
print(tags)

# generate function to convert name and id of tags
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}
print(index2tag)
print(tag2index)

  0%|          | 0/3 [00:00<?, ?it/s]

ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], names_file=None, id=None)
{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [46]:
# hide_output

# create model
import torch

# passing keyword arguments to the from_pretrained() method overrides default values
xlmr_config = AutoConfig.from_pretrained(xlmr_model_name,num_labels=tags.num_classes, id2label=index2tag, label2id=tag2index)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)
xlmr_model = (XLMRobertaForTokenClassification
                .from_pretrained(xlmr_model_name, config=xlmr_config)
                .to(device))

cuda


In [47]:
# tokenize with sentence tokenizer
example_sentence= sentence_list[1]
xlmr_tokens = xlmr_tokenizer(example_sentence).tokens()
input_ids = xlmr_tokenizer.encode(example_sentence, return_tensors="pt")
pd.DataFrame([xlmr_tokens, input_ids[0].numpy()], index=["Tokens", "Input IDs"])

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
Tokens,<s>,▁It,▁is,▁root,ed,▁in,▁our,▁purpose,▁‘,Th,...,▁–,▁for,▁us,▁and,▁our,▁stake,holder,s,.,</s>
Input IDs,0,1650,83,74855,297,23,2446,60042,204,20800,...,46,100,1821,136,2446,145054,31958,7,5,2


In [48]:
# create predictions of NER tags
outputs = xlmr_model(input_ids.to(device)).logits
predictions = torch.argmax(outputs, dim=-1)
print(predictions)
print(f"Number of tokens in sequence: {len(xlmr_tokens)}")
print(f"Shape of outputs: {outputs.shape}")

tensor([[2, 3, 3, 4, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 3, 3, 3, 4, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 6, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 6, 3, 3, 3, 3, 3, 4,
         4, 3, 4, 4, 3, 2]], device='cuda:0')
Number of tokens in sequence: 78
Shape of outputs: torch.Size([1, 78, 7])


In [49]:
# convert predictions to see how BAD random weights are :D
preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
pd.DataFrame([xlmr_tokens, preds], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
Tokens,<s>,▁It,▁is,▁root,ed,▁in,▁our,▁purpose,▁‘,Th,...,▁–,▁for,▁us,▁and,▁our,▁stake,holder,s,.,</s>
Tags,I-PER,B-ORG,B-ORG,I-ORG,B-ORG,I-ORG,I-ORG,B-ORG,B-ORG,B-ORG,...,B-ORG,B-ORG,B-ORG,I-ORG,I-ORG,B-ORG,I-ORG,I-ORG,B-ORG,I-PER


In [50]:
# helper steps for later (aggregating the test steps from before)
def tag_text(text, tags, model, tokenizer):
  # Get tokens with special characters
  tokens = tokenizer(text).tokens()
  # Encode the sequence into IDs
  input_ids = xlmr_tokenizer(text, return_tensors="pt").input_ids.to(device)
  # Get predictions as distribution over 7 possible classes
  outputs = model(input_ids)[0]
  # Take argmax to get most likely class per token
  predictions = torch.argmax(outputs, dim=2)
  # Convert to DataFrame
  preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
  return pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])

**Prepare input data for following training of the model XLMRobertaForTokenClassification (with our config=labels, device, ...)**

In [51]:
# show how labels are given in Pan-X subsets from the XTREME dataset and use the same 
xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME has {len(xtreme_subsets)} configurations")

# PAN-X subsets within XTREME dataset
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
panx_subsets


XTREME has 183 configurations


['PAN-X.af',
 'PAN-X.ar',
 'PAN-X.bg',
 'PAN-X.bn',
 'PAN-X.de',
 'PAN-X.el',
 'PAN-X.en',
 'PAN-X.es',
 'PAN-X.et',
 'PAN-X.eu',
 'PAN-X.fa',
 'PAN-X.fi',
 'PAN-X.fr',
 'PAN-X.he',
 'PAN-X.hi',
 'PAN-X.hu',
 'PAN-X.id',
 'PAN-X.it',
 'PAN-X.ja',
 'PAN-X.jv',
 'PAN-X.ka',
 'PAN-X.kk',
 'PAN-X.ko',
 'PAN-X.ml',
 'PAN-X.mr',
 'PAN-X.ms',
 'PAN-X.my',
 'PAN-X.nl',
 'PAN-X.pt',
 'PAN-X.ru',
 'PAN-X.sw',
 'PAN-X.ta',
 'PAN-X.te',
 'PAN-X.th',
 'PAN-X.tl',
 'PAN-X.tr',
 'PAN-X.ur',
 'PAN-X.vi',
 'PAN-X.yo',
 'PAN-X.zh']

In [52]:
# prepare dataset
panx_ch = defaultdict(DatasetDict)

langs = ["en"] # e.g. ["de","en"]
fracs = [1.0] # e.g. [0.5,0.5]
# panx_ch contains keys for each language and one level lower keys for all splits - here only eng and all data is selected
for lang, frac in zip(langs, fracs):
    # Load monolingual corpus
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    # Shuffle and downsample each split according to spoken proportion
    for split in ds:
        panx_ch[lang][split] = (
            ds[split]
            .shuffle(seed=0)
            .select(range(int(frac * ds[split].num_rows))))
print(panx_ch)
panx_ch_element = panx_ch["en"]["train"][111]
for key, value in panx_ch_element.items():
    print(f"{key}: {value}")

  0%|          | 0/3 [00:00<?, ?it/s]

defaultdict(<class 'datasets.dataset_dict.DatasetDict'>, {'en': DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
})})
tokens: ['Lyon', 'County', ',', 'Nevada']
ner_tags: [5, 6, 6, 6]
langs: ['en', 'en', 'en', 'en']


In [53]:
# show info about tag feature --> we already defined the varibale tags earlier
# --> DatasetDict holds information about all features used (like infos on columns in dataframe)
print('----------features----------------')
print(panx_ch["en"]["train"].features)
print('----------feature ner_tags----------------')
print(panx_ch["en"]["train"].features["ner_tags"].feature)


----------features----------------
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], names_file=None, id=None), length=-1, id=None), 'langs': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}
----------feature ner_tags----------------
ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], names_file=None, id=None)


In [54]:
# examples
print(panx_ch["en"]["train"][4:5])
print(panx_ch["en"]["train"][4:5]["tokens"])
print(panx_ch["en"]["train"][4:5]["ner_tags"])

{'tokens': [['Black', 'Canyon', 'City', ',', 'Arizona']], 'ner_tags': [[5, 6, 6, 6, 6]], 'langs': [['en', 'en', 'en', 'en', 'en']]}
[['Black', 'Canyon', 'City', ',', 'Arizona']]
[[5, 6, 6, 6, 6]]


In [55]:
# the library datasets provides a fast way to tokenize dataset objects with the map() operation.
# the returned input ids need to be augmented with the attention mask and the label ids, that encode the information about with token is associated with each NER tag

def tokenize_and_align_labels(examples):
  tokenized_inputs = xlmr_tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)  # sequence is already split in words
  labels = []
  for idx, label in enumerate(examples["ner_tags"]):
    word_ids = tokenized_inputs.word_ids(batch_index=idx) # get word ids to associate subwords
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
      if word_idx is None or word_idx == previous_word_idx:
        label_ids.append(-100) # -100 is chose, since the PyTorch cross-entropy loss class torch.nn.CrossEntropyLoss has an attribute ignore_index, whose value is -100, which is therefor ignored during training
      else:
        label_ids.append(label[word_idx])
      previous_word_idx = word_idx
    labels.append(label_ids)
  tokenized_inputs["labels"] = labels
  return tokenized_inputs

def encode_panx_dataset(corpus):
  return corpus.map(tokenize_and_align_labels, batched=True,remove_columns=['langs', 'ner_tags', 'tokens'])



In [56]:
# encode dataset
panx_en_encoded = encode_panx_dataset(panx_ch["en"])
panx_en_encoded
print(panx_en_encoded["train"][2])
print(panx_ch["en"]["train"][2])

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'input_ids': [0, 9079, 7113, 202104, 11491, 6, 4, 9079, 7113, 15, 5106, 210298, 1104, 151210, 6, 4, 20271, 30839, 6, 167618, 5106, 1388, 2], 'labels': [-100, 5, -100, 6, 6, 0, -100, 5, -100, 0, 0, -100, -100, -100, 0, -100, 0, 3, 4, -100, 0, 0, -100]}
{'tokens': ['Vaivara', 'concentration', 'camp', ',', 'Vaivara', '(', "''1943–1944", ',', 'during', 'German', 'occupation', "''", ')'], 'ner_tags': [5, 6, 6, 0, 5, 0, 0, 0, 0, 3, 4, 0, 0], 'langs': ['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en']}


**Define performance measures**

In [57]:
# common approach: results for precision, recall, F1-score
# in NER: all words of an entity need to be predicted correctly in order for a prediction to be counted as correct
# libary seqeval: expects predictions and labels as lists of lists 


# during training we need to convert the outputs of the model into such a list that sequeval expects
def align_predictions(predictions, label_ids):
  preds = np.argmax(predictions, axis=2)
  batch_size, seq_len = preds.shape
  labels_list, preds_list = [], []

  for batch_idx in range(batch_size):
    example_labels, example_preds = [], []
    for seq_idx in range(seq_len):
      # Ignore label IDs = -100
      if label_ids[batch_idx, seq_idx] != -100:
        example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
        example_preds.append(index2tag[preds[batch_idx][seq_idx]])
    labels_list.append(example_labels)
    preds_list.append(example_preds)
  return preds_list, labels_list

# helper function for calculation of F1-score for validation set
def compute_metrics(eval_pred):
  y_pred, y_true = align_predictions(eval_pred.predictions, eval_pred.label_ids)
  return {"f1": f1_score(y_true, y_pred),
          "precision": precision_score(y_true, y_pred),
          "recall": recall_score(y_true, y_pred),
          "accuracy": f1_score(y_true, y_pred),
          "classification_report": classification_report(y_true, y_pred, mode='strict', scheme=IOB2)
          }





**Finetuning XLM-RoBERTa and Upload to Huggingface**

In [108]:
num_epochs = 3
batch_size = 24
logging_steps = len(panx_en_encoded["train"]) // batch_size # for panx_en_encoded = 833
model_name = f"{xlmr_model_name}-finetuned-panx-en"
training_args = TrainingArguments(output_dir=model_name, log_level="error", num_train_epochs=num_epochs,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  evaluation_strategy="steps", eval_steps=batch_size, # "steps": Evaluation is done (and logged) every eval_steps. The evaluation strategy to adopt during training.
                                  save_strategy="steps", save_steps=1e6, 
                                  weight_decay=0.01,
                                  disable_tqdm=False, 
                                  logging_strategy="steps", logging_steps=batch_size,  # "epoch": Logging is done at the end of each epoch. The logging strategy to adopt during training.
                                  push_to_hub=True)

In [71]:
notebook_login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [109]:
# this model needs to be crated at huggingface:
f"{xlmr_model_name}-finetuned-panx-en"

'xlm-roberta-base-finetuned-panx-en'

In [110]:
# pad each input sequence to the largest sequence length in a batch
data_collator = DataCollatorForTokenClassification(xlmr_tokenizer)

In [111]:
# init method, to avoid initializing a new model for every Trainer (it loads the untrained model and is called at the beginning of the train() call)
def model_init():
  return (XLMRobertaForTokenClassification
          .from_pretrained(xlmr_model_name, config=xlmr_config)
          .to(device))

In [113]:
trainer = Trainer(model_init=model_init, args=training_args,
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  train_dataset=panx_en_encoded["train"],
                  eval_dataset=panx_en_encoded["validation"],
                  tokenizer=xlmr_tokenizer)

/content/xlm-roberta-base-finetuned-panx-en is already a clone of https://huggingface.co/maren-hugg/xlm-roberta-base-finetuned-panx-en. Make sure you pull the latest changes with `repo.git_pull()`.


In [114]:
trainer.train()

Step,Training Loss,Validation Loss,F1,Precision,Recall,Accuracy,Classification Report
24,1.264400,0.817467,0.421248,0.369598,0.489679,0.421248,precision recall f1-score support LOC 0.28 0.18 0.22 4834 ORG 0.39 0.15 0.22 4677 PER 0.66 0.72 0.69 4635 micro avg 0.49 0.35 0.41 14146 macro avg 0.44 0.35 0.38 14146 weighted avg 0.44 0.35 0.37 14146
48,0.720900,0.563264,0.481697,0.418967,0.566521,0.481697,precision recall f1-score support LOC 0.35 0.49 0.41 4834 ORG 0.47 0.51 0.49 4677 PER 0.77 0.69 0.73 4635 micro avg 0.50 0.56 0.53 14146 macro avg 0.53 0.57 0.54 14146 weighted avg 0.53 0.56 0.54 14146
72,0.595100,0.467027,0.605884,0.558773,0.661671,0.605884,precision recall f1-score support LOC 0.60 0.63 0.61 4834 ORG 0.57 0.51 0.54 4677 PER 0.74 0.81 0.78 4635 micro avg 0.64 0.65 0.65 14146 macro avg 0.64 0.65 0.64 14146 weighted avg 0.64 0.65 0.64 14146
96,0.447500,0.442492,0.665884,0.633618,0.701612,0.665884,precision recall f1-score support LOC 0.65 0.75 0.70 4834 ORG 0.65 0.51 0.57 4677 PER 0.80 0.82 0.81 4635 micro avg 0.70 0.70 0.70 14146 macro avg 0.70 0.70 0.69 14146 weighted avg 0.70 0.70 0.69 14146
120,0.497800,0.446937,0.637484,0.592957,0.689241,0.637484,precision recall f1-score support LOC 0.66 0.68 0.67 4834 ORG 0.60 0.52 0.56 4677 PER 0.72 0.85 0.78 4635 micro avg 0.66 0.68 0.67 14146 macro avg 0.66 0.68 0.67 14146 weighted avg 0.66 0.68 0.67 14146
144,0.438300,0.409320,0.700302,0.666773,0.737382,0.700302,precision recall f1-score support LOC 0.75 0.71 0.73 4834 ORG 0.61 0.63 0.62 4677 PER 0.77 0.86 0.81 4635 micro avg 0.71 0.73 0.72 14146 macro avg 0.71 0.73 0.72 14146 weighted avg 0.71 0.73 0.72 14146
168,0.414800,0.368846,0.712246,0.687660,0.738654,0.712246,precision recall f1-score support LOC 0.71 0.74 0.73 4834 ORG 0.66 0.64 0.65 4677 PER 0.86 0.81 0.84 4635 micro avg 0.74 0.73 0.74 14146 macro avg 0.74 0.73 0.74 14146 weighted avg 0.74 0.73 0.74 14146
192,0.451300,0.369968,0.723576,0.708127,0.739714,0.723576,precision recall f1-score support LOC 0.78 0.71 0.74 4834 ORG 0.66 0.63 0.64 4677 PER 0.84 0.85 0.84 4635 micro avg 0.76 0.73 0.74 14146 macro avg 0.76 0.73 0.74 14146 weighted avg 0.76 0.73 0.74 14146
216,0.378600,0.366571,0.730411,0.712490,0.749258,0.730411,precision recall f1-score support LOC 0.74 0.80 0.77 4834 ORG 0.67 0.61 0.64 4677 PER 0.88 0.81 0.84 4635 micro avg 0.76 0.74 0.75 14146 macro avg 0.77 0.74 0.75 14146 weighted avg 0.77 0.74 0.75 14146
240,0.425000,0.365239,0.704618,0.687374,0.722748,0.704618,precision recall f1-score support LOC 0.74 0.72 0.73 4834 ORG 0.64 0.58 0.61 4677 PER 0.83 0.86 0.84 4635 micro avg 0.74 0.72 0.73 14146 macro avg 0.74 0.72 0.73 14146 weighted avg 0.74 0.72 0.73 14146


Step,Training Loss,Validation Loss,F1,Precision,Recall,Accuracy,Classification Report
24,1.264400,0.817467,0.421248,0.369598,0.489679,0.421248,precision recall f1-score support LOC 0.28 0.18 0.22 4834 ORG 0.39 0.15 0.22 4677 PER 0.66 0.72 0.69 4635 micro avg 0.49 0.35 0.41 14146 macro avg 0.44 0.35 0.38 14146 weighted avg 0.44 0.35 0.37 14146
48,0.720900,0.563264,0.481697,0.418967,0.566521,0.481697,precision recall f1-score support LOC 0.35 0.49 0.41 4834 ORG 0.47 0.51 0.49 4677 PER 0.77 0.69 0.73 4635 micro avg 0.50 0.56 0.53 14146 macro avg 0.53 0.57 0.54 14146 weighted avg 0.53 0.56 0.54 14146
72,0.595100,0.467027,0.605884,0.558773,0.661671,0.605884,precision recall f1-score support LOC 0.60 0.63 0.61 4834 ORG 0.57 0.51 0.54 4677 PER 0.74 0.81 0.78 4635 micro avg 0.64 0.65 0.65 14146 macro avg 0.64 0.65 0.64 14146 weighted avg 0.64 0.65 0.64 14146
96,0.447500,0.442492,0.665884,0.633618,0.701612,0.665884,precision recall f1-score support LOC 0.65 0.75 0.70 4834 ORG 0.65 0.51 0.57 4677 PER 0.80 0.82 0.81 4635 micro avg 0.70 0.70 0.70 14146 macro avg 0.70 0.70 0.69 14146 weighted avg 0.70 0.70 0.69 14146
120,0.497800,0.446937,0.637484,0.592957,0.689241,0.637484,precision recall f1-score support LOC 0.66 0.68 0.67 4834 ORG 0.60 0.52 0.56 4677 PER 0.72 0.85 0.78 4635 micro avg 0.66 0.68 0.67 14146 macro avg 0.66 0.68 0.67 14146 weighted avg 0.66 0.68 0.67 14146
144,0.438300,0.409320,0.700302,0.666773,0.737382,0.700302,precision recall f1-score support LOC 0.75 0.71 0.73 4834 ORG 0.61 0.63 0.62 4677 PER 0.77 0.86 0.81 4635 micro avg 0.71 0.73 0.72 14146 macro avg 0.71 0.73 0.72 14146 weighted avg 0.71 0.73 0.72 14146
168,0.414800,0.368846,0.712246,0.687660,0.738654,0.712246,precision recall f1-score support LOC 0.71 0.74 0.73 4834 ORG 0.66 0.64 0.65 4677 PER 0.86 0.81 0.84 4635 micro avg 0.74 0.73 0.74 14146 macro avg 0.74 0.73 0.74 14146 weighted avg 0.74 0.73 0.74 14146
192,0.451300,0.369968,0.723576,0.708127,0.739714,0.723576,precision recall f1-score support LOC 0.78 0.71 0.74 4834 ORG 0.66 0.63 0.64 4677 PER 0.84 0.85 0.84 4635 micro avg 0.76 0.73 0.74 14146 macro avg 0.76 0.73 0.74 14146 weighted avg 0.76 0.73 0.74 14146
216,0.378600,0.366571,0.730411,0.712490,0.749258,0.730411,precision recall f1-score support LOC 0.74 0.80 0.77 4834 ORG 0.67 0.61 0.64 4677 PER 0.88 0.81 0.84 4635 micro avg 0.76 0.74 0.75 14146 macro avg 0.77 0.74 0.75 14146 weighted avg 0.77 0.74 0.75 14146
240,0.425000,0.365239,0.704618,0.687374,0.722748,0.704618,precision recall f1-score support LOC 0.74 0.72 0.73 4834 ORG 0.64 0.58 0.61 4677 PER 0.83 0.86 0.84 4635 micro avg 0.74 0.72 0.73 14146 macro avg 0.74 0.72 0.73 14146 weighted avg 0.74 0.72 0.73 14146


TrainOutput(global_step=2502, training_loss=0.26952576098872794, metrics={'train_runtime': 1297.3455, 'train_samples_per_second': 46.248, 'train_steps_per_second': 1.929, 'total_flos': 1178415902403696.0, 'train_loss': 0.26952576098872794, 'epoch': 3.0})

In [115]:
trainer.push_to_hub(commit_message="Training completed!")

Upload file pytorch_model.bin:   0%|          | 1.00/1.03G [00:00<?, ?B/s]

Upload file sentencepiece.bpe.model:   0%|          | 1.00/4.83M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/2.81k [00:00<?, ?B/s]

Upload file runs/Apr14_14-04-19_7ded55b56291/1681481133.6566057/events.out.tfevents.1681481133.7ded55b56291.48…

Upload file runs/Apr14_14-04-19_7ded55b56291/events.out.tfevents.1681481133.7ded55b56291.48074.12:   0%|      …

To https://huggingface.co/maren-hugg/xlm-roberta-base-finetuned-panx-en
   902b70b..2ce5da4  main -> main

   902b70b..2ce5da4  main -> main

To https://huggingface.co/maren-hugg/xlm-roberta-base-finetuned-panx-en
   2ce5da4..1de6d86  main -> main

   2ce5da4..1de6d86  main -> main



'https://huggingface.co/maren-hugg/xlm-roberta-base-finetuned-panx-en/commit/2ce5da4f205928dfdfe761485ed0931a9b4c5088'

In [116]:
metrics_train=trainer.state.log_history
for item in metrics_train:
  print(item)

{'loss': 1.2644, 'learning_rate': 4.9520383693045564e-05, 'epoch': 0.03, 'step': 24}
{'eval_loss': 0.8174670934677124, 'eval_f1': 0.4212478715640963, 'eval_precision': 0.3695976950165404, 'eval_recall': 0.48967906121871907, 'eval_accuracy': 0.4212478715640963, 'eval_classification_report': '              precision    recall  f1-score   support\n\n         LOC       0.28      0.18      0.22      4834\n         ORG       0.39      0.15      0.22      4677\n         PER       0.66      0.72      0.69      4635\n\n   micro avg       0.49      0.35      0.41     14146\n   macro avg       0.44      0.35      0.38     14146\nweighted avg       0.44      0.35      0.37     14146\n', 'eval_runtime': 10.5164, 'eval_samples_per_second': 950.892, 'eval_steps_per_second': 39.652, 'epoch': 0.03, 'step': 24}
{'loss': 0.7209, 'learning_rate': 4.904076738609113e-05, 'epoch': 0.06, 'step': 48}
{'eval_loss': 0.5632641315460205, 'eval_f1': 0.48169742141011, 'eval_precision': 0.4189669594312003, 'eval_reca